In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("../input/new-york-city-taxi-fare-prediction/train.csv", nrows = 1_000_000)
test = pd.read_csv("../input/new-york-city-taxi-fare-prediction/test.csv")
sample_submission = pd.read_csv("../input/new-york-city-taxi-fare-prediction/sample_submission.csv")

In [ ]:
train.isnull().sum()

In [ ]:
## NaN レコードを削除
train.dropna(inplace=True)

In [ ]:
train.describe()

In [ ]:
train.query('passenger_count > 6')

In [ ]:
train.query('passenger_count < 1')

In [ ]:
train.query('fare_amount < 0')

In [ ]:
train.query('pickup_longitude < -180 or pickup_longitude > 180')

In [ ]:
train.query('dropoff_longitude < -180 or dropoff_longitude > 180')

In [ ]:
train.query('pickup_latitude < -90 or pickup_latitude > 90')

In [ ]:
train.query('dropoff_latitude < -90 or dropoff_latitude > 90')

In [ ]:
# 外れ値を削除する
# fare_amount は、マイナス料金のタクシーは外れ値として扱う
# passenger_count は、誰も乗らないまたは7人以上乗るタクシーは外れ値として扱う
# pickup_longitude/dropoff_longitude は、経度が-180度または180度を超える場所で乗り降りするタクシーは外れ値として扱う
# pickup_latitude/dropoff_latitude は、緯度が-90度または90度を超える場所で乗り降りするタクシーは外れ値として扱う
train = train.query('1 <= passenger_count <= 6 and '
                    '0 <= fare_amount and'
                    '-180 <= pickup_longitude <= 180 and'
                    '-180 <= dropoff_longitude <= 180 and'
                    '-90 <= pickup_latitude <= 90 and'
                    '-90 <= dropoff_latitude <= 90'
                   )
train.describe()

In [ ]:
# index の振りなおし
train.reset_index(drop=True, inplace=True)
train

In [ ]:
data = pd.concat([train, test], sort=False)

In [ ]:
data.head()

In [ ]:
# 日付から、曜日を計算して追加
import datetime as dt
data['weekday'] = data.apply(lambda x: dt.date(int(x['pickup_datetime'][0:4]), int(x['pickup_datetime'][5:7]), int(x['pickup_datetime'][8:10])).weekday(), axis=1)

data.head()

In [ ]:
# pickup_datatime は扱いづらいため削除（key に乗った時刻が同じようにあるため、削除しても問題ない）
# key を数値に変換して扱えるようにする
data = data.drop('pickup_datetime', axis=1)
data['key'] = data['key'].str.replace('[- :]', '')
data['key'] = data['key'].astype(float)

data.head()

In [ ]:
# 緯度経度から、距離を計算してカラムに追加
from geopy import distance
data['distance'] = data.apply(lambda x: distance.distance((x['pickup_latitude'],x['pickup_longitude']), (x['dropoff_latitude'],x['dropoff_longitude'])).miles, axis=1)

data.head()

In [ ]:
train = data[:len(train)]
test = data[len(train):]

y_train = train['fare_amount']
X_train = train.drop('fare_amount', axis=1)
X_test = test.drop('fare_amount', axis=1)

X_train.head()

In [ ]:
from sklearn.model_selection import KFold

y_preds = []
models = []
oof_train = np.zeros((len(X_train),))
cv = KFold(n_splits=5, shuffle=True, random_state=0)

categorical_features = []

In [ ]:
import lightgbm as lgb

params = {
    'objective': 'regression',
    'max_bin': 300,
    'learning_rate': 0.05,
    'num_leaves': 40,
}

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train, y_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]

    lgb_train = lgb.Dataset(X_tr, y_tr,
                            categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_val, y_val,
                           reference=lgb_train,
                           categorical_feature=categorical_features)

    model = lgb.train(params, lgb_train,
                      valid_sets=[lgb_train, lgb_eval],
                      verbose_eval=10,
                      num_boost_round=1000,
                      early_stopping_rounds=10)

    oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    y_preds.append(y_pred)
    models.append(model)

In [ ]:
pd.DataFrame(oof_train).to_csv('oof_train_kfold.csv', index=False)

scores = [
    m.best_score['valid_1']['l2'] for m in models
]
score = sum(scores) / len(scores)
print('===CV scores===')
print(scores)
print(score)

In [ ]:
from sklearn.metrics import mean_squared_error

y_pred_oof = oof_train
np.sqrt(mean_squared_error(y_train, y_pred_oof))

In [ ]:
len(y_preds)

In [ ]:
y_preds[0][:10]

In [ ]:
y_sub = sum(y_preds) / len(y_preds)
y_sub[:10]

In [ ]:
sub_lgb = sample_submission

sub_lgb['fare_amount'] = y_sub
sub_lgb.to_csv("submission_lightgbm.csv", index=False)

sub_lgb.head()